# **YOGA POSE CLASSIFICATION [COMPARING WITH OTHER MODELS]**

BY

        Anand K S [AM.EN.U4CSE19106]
        Bharath Prathap Nair [AM.EN.U4CSE19113]
        Rahan Manoj [AM.EN.U4CSE19144]

This notebook contains the code used to generate various CNN architectures that are then compared with the efficient net architecture.

The Codes used while implementing these different architectures are given in this notebook.

The dataset used in this notebook is taken from [Kaggle](www.kaggle.com/shrutisaxena/yoga-pose-image-classification-dataset). The creator of the dataset is Shruthi Saxena. The dataset comprises of 107 classes/yoga-poses and 5991 images.


**IMPORTING LIBRARIES**

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import PIL
from PIL import Image
from tqdm import tqdm                                
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
import warnings
warnings.filterwarnings('ignore')

# **LOADING THE DATASET**

In [ ]:
from google.colab import drive  
drive._mount('/content/drive') 

Mounted at /content/drive


In [ ]:
data_dir = '/content/drive/MyDrive/Colab Notebooks/Yoga_image/augged'
labels = os.listdir(data_dir)
for i in labels:
  l = os.listdir(data_dir+'/'+i)
  print(i, len(l))


hanumanasana 70
halasana 132
ustrasana 174
dwi pada viparita dandasana 110
janu sirsasana 96
bhujangasana 146
mayurasana 102
dandasana 120
simhasana 98
bitilasana 174
ananda balasana 118
gomukhasana 144
makarasana 114
salamba sirsasana 120
chakravakasana 140
virabhadrasana iii 122
parsva bakasana 112
ganda bherundasana 78
supta baddha konasana 142
anjaneyasana 128
parivrtta parsvakonasana 80
urdhva dhanurasana 136
eka pada koundinyanasana ii 116
eka pada koundinyanasana i 102
utthita trikonasana 138
durvasasana 78
upavistha konasana 120
viparita karani 138
virabhadrasana ii 111
salamba bhujangasana 110
yoganidrasana 92
bharadvajasana i 108
adho mukha svanasana 137
eka pada rajakapotasana 88
sukhasana 100
matsyasana 114
vasisthasana 148
astavakrasana 144
paripurna navasana 136
marichyasana i 98
agnistambhasana 66
virabhadrasana i 109
tulasana 62
kurmasana 80
marichyasana iii 66
marjaryasana 92
camatkarasana 108
adho mukha vriksasana 118
tolasana 120
salamba sarvangasana 134
parivrtta ja

In [ ]:
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms

transformations = transforms.Compose([transforms.Resize((256, 256)), 
                                      transforms.RandomCrop(224),
                                     transforms.ToTensor(),
                                      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

dataset = ImageFolder(data_dir, transform = transformations)

In [ ]:
dataset.class_to_idx                                          # Shows index of each label for prediction purposes

{'adho mukha svanasana': 0,
 'adho mukha vriksasana': 1,
 'agnistambhasana': 2,
 'ananda balasana': 3,
 'anantasana': 4,
 'anjaneyasana': 5,
 'ardha bhekasana': 6,
 'ardha chandrasana': 7,
 'ardha matsyendrasana': 8,
 'ardha pincha mayurasana': 9,
 'ardha uttanasana': 10,
 'ashtanga namaskara': 11,
 'astavakrasana': 12,
 'baddha konasana': 13,
 'bakasana': 14,
 'balasana': 15,
 'bhairavasana': 16,
 'bharadvajasana i': 17,
 'bhekasana': 18,
 'bhujangasana': 19,
 'bhujapidasana': 20,
 'bitilasana': 21,
 'camatkarasana': 22,
 'chakravakasana': 23,
 'chaturanga dandasana': 24,
 'dandasana': 25,
 'dhanurasana': 26,
 'durvasasana': 27,
 'dwi pada viparita dandasana': 28,
 'eka pada koundinyanasana i': 29,
 'eka pada koundinyanasana ii': 30,
 'eka pada rajakapotasana': 31,
 'eka pada rajakapotasana ii': 32,
 'ganda bherundasana': 33,
 'garbha pindasana': 34,
 'garudasana': 35,
 'gomukhasana': 36,
 'halasana': 37,
 'hanumanasana': 38,
 'janu sirsasana': 39,
 'kapotasana': 40,
 'krounchasana': 

In [ ]:
targets = dataset.targets
len(targets) # After augmentation, the size of the dataset has increased as can be seen from its output. [11899]

11901

# **TRAIN - TEST SPLIT**

20% of the data is used for testing, while the remaining 80% is used for training. A stratified split was done so that the number of classes in training and test sets are split uniformly.

In [ ]:
from sklearn.model_selection import train_test_split
train_idx, test_idx= train_test_split(np.arange(len(targets)),test_size=0.20,shuffle=True,stratify=targets) #train_idx, test_idx gives the indexes of the data in each split

In [ ]:
trainloader = torch.utils.data.DataLoader(dataset, batch_size=32, sampler=train_idx)    # Based on the indexes we get after splitting in the above sets, we load the corresponding data 
testloader = torch.utils.data.DataLoader(dataset, batch_size=32, sampler=test_idx)      # into trainloader and testloader.



# **RESNET**

In [ ]:
resnet = models.resnet18(pretrained=True)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [ ]:
resnet = resnet.to(device)
loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adagrad(resnet.parameters(), lr=0.01)

In [ ]:
train_losses = []
train_acc = []
test_losses = []
test_acc = []

In [ ]:
for epoch in range(8):   # Run for loop for number of times. The number of times is the epochs
    train_correct = 0
    total1 = 0
    running_loss = 0
    with tqdm(trainloader, unit="batch") as tepoch:
        for images, labels in tepoch:                               # Training the dataset
            tepoch.set_description(f"Training : Epoch {epoch+1}")
            images, labels = images.cuda(), labels.cuda()
            labels = labels.long()
            optimizer.zero_grad()
            outputs = resnet(images) 
            loss = loss_func(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total1 += labels.size(0)

            if torch.cuda.is_available():
                train_correct += (predicted.cpu() == labels.cpu()).sum().item()
                traincorrect = (predicted.cpu() == labels.cpu()).sum().item()
            else:
                train_correct += (predicted == labels).sum().item()
                traincorrect = (predicted == labels).sum().item()
            tepoch.set_postfix(loss=loss.item(), accuracy=100* (traincorrect/32.0))

    train_accuracy = 100 * train_correct / total1
    train_loss = running_loss/len(trainloader)
    train_acc.append(train_accuracy)
    train_losses.append(train_loss)


    test_loss = 0
    correct = 0
    total = 0
    pred = []
    test = []
    with tqdm(testloader, unit="batch") as tstepoch:      
        for images, labels in tstepoch:                                       # Testing 
            tstepoch.set_description(f"Testing : Epoch {epoch+1}")
            images, labels = images.cuda(), labels.cuda()
            labels = labels.long() 
            outputs = resnet(images)
            loss_t = loss_func(outputs, labels)
            test_loss += loss_t.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            if torch.cuda.is_available():
                correct += (predicted.cpu() == labels.cpu()).sum().item()
                tstcorrect = (predicted.cpu() == labels.cpu()).sum().item()
            else:
                correct += (predicted == labels).sum().item()
                tstcorrect = (predicted == labels).sum().item()
            tstepoch.set_postfix(loss=loss_t.item(), accuracy=100*(tstcorrect/32.0))
 
    test_loss=test_loss/len(testloader)
    accuracy = 100 * correct / total
    test_acc.append(accuracy)
    test_losses.append(test_loss)
    print('\nEpoch: {} - loss: {} - accuracy: {} - val_Loss: {} - val_accuracy: {}\n'.format(epoch+1, train_loss, train_accuracy, test_loss, accuracy))

Testing : Epoch 1: 100%|██████████| 75/75 [00:32<00:00,  2.32batch/s, accuracy=0, loss=4.25]



Epoch: 1 - loss: 4.651851864469132 - accuracy: 3.203781512605042 - val_Loss: 4.125290314356486 - val_accuracy: 5.585888282234356



Testing : Epoch 2: 100%|██████████| 75/75 [00:32<00:00,  2.30batch/s, accuracy=3.12, loss=3.16]



Epoch: 2 - loss: 3.417779590459478 - accuracy: 16.19747899159664 - val_Loss: 2.9291252581278484 - val_accuracy: 25.829483410331793



Testing : Epoch 3: 100%|██████████| 75/75 [00:32<00:00,  2.31batch/s, accuracy=15.6, loss=2.61]


...


Epoch: 6 - loss: 0.7449280785634214 - accuracy: 79.00210084033614 - val_Loss: 1.610352461338043 - val_accuracy: 56.656866862662746



Testing : Epoch 7: 100%|██████████| 75/75 [00:31<00:00,  2.34batch/s, accuracy=25, loss=1.57]



Epoch: 7 - loss: 0.46755392839444565 - accuracy: 87.51050420168067 - val_Loss: 1.5786148182551065 - val_accuracy: 57.790844183116334



Testing : Epoch 8: 100%|██████████| 75/75 [00:31<00:00,  2.35batch/s, accuracy=25, loss=1.61]


Epoch: 8 - loss: 0.27052286207275905 - accuracy: 93.64495798319328 - val_Loss: 1.5812443240483602 - val_accuracy: 59.68080638387232



# **VGG**

In [ ]:
from torchvision import models
vgg = models.vgg16_bn()

In [ ]:
final_in_features = vgg.classifier[6].in_features
mod_classifier = list(vgg.classifier.children())[:-1]
mod_classifier.extend([nn.Linear(final_in_features, 107)])
print(mod_classifier)

[Linear(in_features=25088, out_features=4096, bias=True), ReLU(inplace=True), Dropout(p=0.5, inplace=False), Linear(in_features=4096, out_features=4096, bias=True), ReLU(inplace=True), Dropout(p=0.5, inplace=False), Linear(in_features=4096, out_features=107, bias=True)]


In [ ]:
vgg.classifier = nn.Sequential(*mod_classifier)

In [ ]:
def evaluation(dataloader, model):
    total, correct = 0, 0
    for data in dataloader:
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, pred = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (pred == labels).sum().item()
    return 100 * correct / total

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [ ]:
vgg = vgg.to(device)
loss_fn = nn.CrossEntropyLoss()
opt = optim.SGD(vgg.parameters(), lr=0.05)

In [ ]:
train_losses = []
train_acc = []
test_losses = []
test_acc = []
for epoch in range(8):   # Run for loop for number of times. The number of times is the epochs
    train_correct = 0
    total1 = 0
    running_loss = 0
    with tqdm(trainloader, unit="batch") as tepoch:
        for images, labels in tepoch:                               # Training the dataset
            tepoch.set_description(f"Training : Epoch {epoch+1}")
            images, labels = images.cuda(), labels.cuda()
            labels = labels.long()
            opt.zero_grad()
            outputs = vgg(images) 
            loss = loss_fn(outputs, labels)
            loss.backward()
            opt.step()
            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total1 += labels.size(0)

            if torch.cuda.is_available():
                train_correct += (predicted.cpu() == labels.cpu()).sum().item()
                traincorrect = (predicted.cpu() == labels.cpu()).sum().item()
            else:
                train_correct += (predicted == labels).sum().item()
                traincorrect = (predicted == labels).sum().item()
            tepoch.set_postfix(loss=loss.item(), accuracy=100* (traincorrect/32.0))

    train_accuracy = 100 * train_correct / total1
    train_loss = running_loss/len(trainloader)
    train_acc.append(train_accuracy)
    train_losses.append(train_loss)


    test_loss = 0
    correct = 0
    total = 0
    pred = []
    test = []
    with tqdm(testloader, unit="batch") as tstepoch:      
        for images, labels in tstepoch:                                       # Testing 
            tstepoch.set_description(f"Testing : Epoch {epoch+1}")
            images, labels = images.cuda(), labels.cuda()
            labels = labels.long() 
            outputs = vgg(images)
            loss_t = loss_fn(outputs, labels)
            test_loss += loss_t.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            if torch.cuda.is_available():
                correct += (predicted.cpu() == labels.cpu()).sum().item()
                tstcorrect = (predicted.cpu() == labels.cpu()).sum().item()
            else:
                correct += (predicted == labels).sum().item()
                tstcorrect = (predicted == labels).sum().item()
            tstepoch.set_postfix(loss=loss_t.item(), accuracy=100*(tstcorrect/32.0))
 
    test_loss=test_loss/len(testloader)
    accuracy = 100 * correct / total
    test_acc.append(accuracy)
    test_losses.append(test_loss)
    print('\nEpoch: {} - loss: {} - accuracy: {} - val_Loss: {} - val_accuracy: {}\n'.format(epoch+1, train_loss, train_accuracy, test_loss, accuracy))

Testing : Epoch 1: 100%|██████████| 75/75 [00:51<00:00,  1.45batch/s, accuracy=12.5, loss=3.1]



Epoch: 1 - loss: 1.685742678258243 - accuracy: 52.305914486815844 - val_Loss: 2.5152920802434284 - val_accuracy: 38.69747899159664



Testing : Epoch 2: 100%|██████████| 75/75 [00:52<00:00,  1.44batch/s, accuracy=18.8, loss=2.66]



Epoch: 2 - loss: 1.5893370541150138 - accuracy: 53.93423678957874 - val_Loss: 2.3241611909866333 - val_accuracy: 41.21848739495798



Testing : Epoch 3: 100%|██████████| 75/75 [00:51<00:00,  1.46batch/s, accuracy=12.5, loss=3.07]


...


Epoch: 6 - loss: 1.15396507884432 - accuracy: 65.11188150015758 - val_Loss: 2.348325098355611 - val_accuracy: 45.79831932773109



Testing : Epoch 7: 100%|██████████| 75/75 [00:52<00:00,  1.44batch/s, accuracy=12.5, loss=2.93]



Epoch: 7 - loss: 1.099222873081297 - accuracy: 66.80323563399517 - val_Loss: 2.279411981900533 - val_accuracy: 45.08403361344538



Testing : Epoch 8: 100%|██████████| 75/75 [00:52<00:00,  1.44batch/s, accuracy=9.38, loss=3.32]


Epoch: 8 - loss: 1.035849339709986 - accuracy: 68.02185103477257 - val_Loss: 2.2568754291534425 - val_accuracy: 46.30252100840336



# **MOBILE NET**

In [ ]:
mobNet = models.mobilenet_v2(pretrained=True)

In [ ]:
mobNet.classifier[-1].out_features = 107

In [ ]:
mobNet.classifier[-1]

Linear(in_features=1280, out_features=107, bias=True)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
mobNet = mobNet.to('cuda')    # Load the model to GPU.

In [ ]:
optimizer = optim.Adadelta(mobNet.parameters())     # Defining the optimiser.
loss_func = nn.CrossEntropyLoss()                            # Defining the loss function
                                                    # Vary optimiser and loss function as per requirement while training.

In [ ]:
train_losses = []
train_acc = []
test_losses = []
test_acc = []
for epoch in range(10):   # Run for loop for number of times. The number of times is the epochs
    train_correct = 0
    total1 = 0
    running_loss = 0
    with tqdm(trainloader, unit="batch") as tepoch:
        for images, labels in tepoch:                               # Training the dataset
            tepoch.set_description(f"Training : Epoch {epoch+1}")
            images, labels = images.cuda(), labels.cuda()
            labels = labels.long()
            optimizer.zero_grad()
            outputs = mobNet(images) 
            loss = loss_func(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total1 += labels.size(0)

            if torch.cuda.is_available():
                train_correct += (predicted.cpu() == labels.cpu()).sum().item()
                traincorrect = (predicted.cpu() == labels.cpu()).sum().item()
            else:
                train_correct += (predicted == labels).sum().item()
                traincorrect = (predicted == labels).sum().item()
            tepoch.set_postfix(loss=loss.item(), accuracy=100* (traincorrect/32.0))

    train_accuracy = 100 * train_correct / total1
    train_loss = running_loss/len(trainloader)
    train_acc.append(train_accuracy)
    train_losses.append(train_loss)


    test_loss = 0
    correct = 0
    total = 0
    pred = []
    test = []
    with tqdm(testloader, unit="batch") as tstepoch:      
        for images, labels in tstepoch:                                       # Testing 
            tstepoch.set_description(f"Testing : Epoch {epoch+1}")
            images, labels = images.cuda(), labels.cuda()
            labels = labels.long() 
            outputs = mobNet(images)
            loss_t = loss_func(outputs, labels)
            test_loss += loss_t.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            if torch.cuda.is_available():
                correct += (predicted.cpu() == labels.cpu()).sum().item()
                tstcorrect = (predicted.cpu() == labels.cpu()).sum().item()
            else:
                correct += (predicted == labels).sum().item()
                tstcorrect = (predicted == labels).sum().item()
            tstepoch.set_postfix(loss=loss_t.item(), accuracy=100*(tstcorrect/32.0))
 
    test_loss=test_loss/len(testloader)
    accuracy = 100 * correct / total
    test_acc.append(accuracy)
    test_losses.append(test_loss)
    print('\nEpoch: {} - loss: {} - accuracy: {} - val_Loss: {} - val_accuracy: {}\n'.format(epoch+1, train_loss, train_accuracy, test_loss, accuracy))

Testing : Epoch 1: 100%|██████████| 75/75 [07:42<00:00,  6.16s/batch, accuracy=0, loss=3.36]



Epoch: 1 - loss: 3.8816117800322156 - accuracy: 13.803971005357706 - val_Loss: 3.0033382511138917 - val_accuracy: 24.07563025210084



Testing : Epoch 2: 100%|██████████| 75/75 [00:34<00:00,  2.19batch/s, accuracy=9.38, loss=3.13]



Epoch: 2 - loss: 2.4884316837227582 - accuracy: 34.14224183212522 - val_Loss: 2.5727002255121865 - val_accuracy: 34.95798319327731



Testing : Epoch 3: 100%|██████████| 75/75 [00:32<00:00,  2.34batch/s, accuracy=9.38, loss=2.7]


...


Epoch: 8 - loss: 1.1323811785886753 - accuracy: 67.03435234793571 - val_Loss: 1.7036404911677043 - val_accuracy: 55.46218487394958



Testing : Epoch 9: 100%|██████████| 75/75 [00:32<00:00,  2.28batch/s, accuracy=15.6, loss=3.1]



Epoch: 9 - loss: 1.0451367997483119 - accuracy: 68.77823300766887 - val_Loss: 1.7924599854151408 - val_accuracy: 55.54621848739496



Testing : Epoch 10: 100%|██████████| 75/75 [00:33<00:00,  2.26batch/s, accuracy=18.8, loss=1.63]


Epoch: 10 - loss: 0.9687209528364591 - accuracy: 70.59565080365584 - val_Loss: 1.718844230969747 - val_accuracy: 54.411764705882355

